In [1]:
!pip install opencv-python==4.4.0.40

     |████████████████████████████████| 49.4MB 85kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


In [2]:
!pip install scikit-image

In [3]:
!pip install pixellib

     |████████████████████████████████| 71kB 4.6MB/s 
  Created wheel for imantics: filename=imantics-0.1.12-cp37-none-any.whl size=16033 sha256=e1a6f0765cf888642c1f9c39084ec5135b09a8f69ed5ebdd6b6c735bf0af252a
  Stored in directory: /root/.cache/pip/wheels/73/93/1c/9e2fc52eb74441941bc76cac441ddcc2c7ad67b18e1849e62a
Successfully built imantics


In [8]:
import pixellib
from pixellib.instance import instance_segmentation
import matplotlib.pyplot as plt
import cv2

class Measure:
  #Class vars
  #stores the pixel count of the desired area
  PIXEL_COUNT = 0

    #Input from user
  def __init__(self, input_im, a_height, a_leglength):
    #actual height of the person measured manually
    self.a_height = a_height
    #actual leg length of the person(below belly button) measured manually
    self.a_leglength = a_leglength
    #ratio of headtoheight : 1:7.5 approx 
    self.a_head = (1/7.5)*a_height  #actual size of head

    #Detect object(person) from image
    instance_seg = instance_segmentation()
    #mask_rcnn_coco.h5 is the pre trained model for object detection
    instance_seg.load_model("/content/drive/MyDrive/mask_rcnn_coco.h5")
    self.segmask, output = instance_seg.segmentImage(input_im, show_bboxes= True, output_image_name = "final.jpg")

   #converts the coloured image to silhouette
  def segment_to_black(self):
     #Reading the image generated after person detection 
    im = cv2.imread('/content/final.jpg')
    #generates a rectangle around detected person in the image
    for index_x, x in enumerate(im):
        for index_y, y in enumerate(x):
          #stores the coordinates of the rectangle generated around the person in the image
          v = self.segmask['masks'][index_x][index_y] 
          #if person is detected ,it is converted into a silhouette
          if v.any() == [True]:
            im[index_x][index_y] = [0, 0, 0] 
    cv2.imwrite("/content/final.jpg", im)   # saving and overwriting final.jpg 
    #plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB)) #graphical representation

  def crop_and_resize(self):
    #Cropping the final.jpg 
    final = cv2.imread("/content/final.jpg")   
    x1, y1, x2, y2 = self.segmask['rois'][0]
    crop_img = final[x1:x2, y1:y2]
    cv2.imwrite("finalcrop.jpg", crop_img)
    print(f"Cropped image shape - {crop_img.shape}") #prints the coordinates of the cropped image
    #plt.imshow(cv2.cvtColor(cv2.imread("/content/finalcrop.jpg"), cv2.COLOR_BGR2RGB))

    #Resizing the final image
    i_xAxis = 64.0 / crop_img.shape[1]  #setting one parameter(width) along x-axis
    dim = (64, int(crop_img.shape[0] * i_xAxis))  #stores the x and y coordinates of the cropped image
    self.resized = cv2.resize(crop_img, dim, interpolation = cv2.INTER_AREA)
    cv2.imwrite("resized.jpg", self.resized)
    print(f"Resized image shape - {self.resized.shape}")
    #plt.imshow(cv2.cvtColor(cv2.imread("/content/resized.jpg"), cv2.COLOR_BGR2RGB))
  
    #for front and side hip measurement
  def hip_measurement(self): 
    #calculates the ratio of upper body to height
    rubth = (self.a_height - self.a_leglength)/self.a_height
    #stores the height of upper body in the image
    i_ub = rubth * self.resized.shape[0] 
    #stores the height of lower body in the image
    i_recub = self.resized.shape[0] - i_ub
    #generates a green coloured rectangle around the calculated dimensions(hip region)
    flood_input = cv2.rectangle(self.resized, (0,int(i_ub)), (64,int(i_recub)),(0,255,0)) 
    x1, y1, x2, y2 = 0, int(i_ub), 64, int(i_recub)
    #calculates the mid point of the hip region
    mid_y = y1 + int((y2 - y1)/2) 
    mid_x = int((x2 - x1)/2)
    print(f"Hip detection rectangle mid-point -> {mid_x, mid_y}")
    return (mid_x, mid_y, flood_input)

  def shoulder_measurement(self):
    ratio_ubth = (self.a_height - self.a_leglength)/(self.a_height)
    i_head = self.resized.shape[0]*(1/7.5)
    i_upperbody = (7.5*ratio_ubth)*(i_head)
    fub = i_head + ((i_upperbody-i_head)/2)
    flood_input = cv2.rectangle(self.resized, (0,int(i_head)),(64,(int(fub))), (0,255,0))
    x1, y1, x2, y2 = 0, int(i_head), 64, int(fub)
    mid_y = y1 + int((y2 - y1)/2)
    mid_x = int((x2 - x1)/2)
    print(f"Shoulder detection rectangle mid-point -> {mid_x, mid_y}")
    return (mid_x, mid_y, flood_input)
     
    #for side chest measurement
  def side_chest_measurement(self):
    #ratio of upper body to height
    ratio_ubth = (self.a_height - self.a_leglength)/(self.a_height)
    #stores the size of head
    i_head = self.resized.shape[0]*(1/7.5)
    i_upperbody = (7.5*ratio_ubth)*(i_head)
    fub = i_head + ((i_upperbody-i_head)/2)

    x1, y1, x2, y2 = 0, int(i_head), 64, int(fub)
    #claculating the mid point of chest region
    mid_y = y1 + int((y2 - y1)/2)
    mid_x = int((x2 - x1)/2)
    a = i_head + ((int(fub)-mid_y)*2.2)
    mid_x = 32
    mid_y = int(a)
    print(f"Side-Chest detection mid-point -> {mid_x, mid_y}")
    return (mid_x, mid_y)

  #for front chest measurements
  def front_chest_measurement(self):
    #ratio of upper body to the height
    ratio_ubth = (self.a_height - self.a_leglength)/(self.a_height)
    i_head = self.resized.shape[0]*(1/7.5)
    i_upperbody = (7.5*ratio_ubth)*(i_head)
    fub = i_head + ((i_upperbody-i_head)/2)

    x1, y1, x2, y2 = 0, int(i_head), 64, int(fub)
    mid_y = y1 + int((y2 - y1)/2)
    mid_x = int((x2 - x1)/2)
    a = i_head + ((int(fub)-mid_y)*2)
    mid_x = 32
    mid_y = int(a)
    print(f"Front-Chest detection mid-point -> {mid_x, mid_y}")
    return (mid_x, mid_y)

  #area(desired body parts) filling function
  def flood_fill_right(self, x, y, im):
    OldColor = [0, 0, 0] #black
    CurrentColor = im[y][x]
    #color = [b,g,r]
    
    #Main algo -> if CurrentColor == OldColor
    if (CurrentColor[0] == OldColor[0] and CurrentColor[1] == OldColor[1] and CurrentColor[2] == OldColor[2]):
        im = cv2.circle(im, (x, y), radius=0, color=(0, 0, 255), thickness = -1) #draws a red line in the desired black region(hip/chest)
        #calculates the pixel count of the desired region
        Measure.PIXEL_COUNT += 1
        #traversing to the right side of the image
        self.flood_fill_right(x+1, y,im)

  def flood_fill_left(self, x, y, im):
      OldColor = [0, 0, 0] #black
      CurrentColor = im[y][x-1]
      # color = [b, g, r]
      
      #Main algo -> if CurrentColor == OldColor
      if (CurrentColor[0] == OldColor[0] and CurrentColor[1] == OldColor[1] and CurrentColor[2] == OldColor[2]):
          im = cv2.circle(im, (x, y), radius=0, color=(0, 0, 255), thickness = -1)
          Measure.PIXEL_COUNT += 1
          #traversing to the left side of the image
          self.flood_fill_left(x-1, y,im)
      return im

In [10]:
height = float(input("Enter height: "))
leglength = float(input("Enter leglength: "))

Enter height: 162.5
Enter leglength: 101.6


In [11]:
#Front Image
person = Measure("/content/testa2.jpeg" , height, leglength) #(image_for_measurement,actual height,actual leg length)
person.segment_to_black() #silhouette formation

person.crop_and_resize() #cropping and resizing of image

x, y, flood_input = person.hip_measurement() #for counting pixel of front hip measurement
Measure.PIXEL_COUNT = 0
person.flood_fill_right(x, y, flood_input)
im = person.flood_fill_left(x, y, flood_input)
cv2.imwrite("fronthip.jpeg",im)
fronthip = Measure.PIXEL_COUNT + 4
print(f"Hip Pixel count -> {Measure.PIXEL_COUNT + 4}")

person.crop_and_resize()

x, y = person.front_chest_measurement() #for counting of front chest measurement
Measure.PIXEL_COUNT = 0
person.flood_fill_right(x, y, person.resized)
im = person.flood_fill_left(x, y, person.resized)
cv2.imwrite("frontchest.jpeg",im)
frontchest = Measure.PIXEL_COUNT + 2
print(f"Front Chest Pixel count -> {Measure.PIXEL_COUNT + 2}")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Processed image saved successfully in your current working directory.
Cropped image shape - (797, 785, 3)
Resized image shape - (64, 64, 3)
Hip detection rectangle mid-point -> (32, 31)
Hip Pixel count -> 16
Cropped image shape - (797, 785, 3)
Resized image shape - (64, 64, 3)
Front-Chest detection mid-point -> (32, 16)
Front Chest Pixel count -> 19


In [12]:
# Side Image
person = Measure("/content/testa3.jpeg",height ,leglength )
person.segment_to_black() #silhouette conversion

person.crop_and_resize() #cropping and resizing the image

x, y, flood_input = person.hip_measurement() #for counting pixel of side hip measurement
Measure.PIXEL_COUNT = 0
person.flood_fill_right(x, y, flood_input)
im = person.flood_fill_left(x, y, flood_input)
cv2.imwrite("sidehip.jpeg",im)
sidehip = Measure.PIXEL_COUNT + 2
print(f"Hip Pixel count -> {Measure.PIXEL_COUNT + 2}")

person.crop_and_resize()

x, y = person.side_chest_measurement()  #for counting pixel of side chest measurement
Measure.PIXEL_COUNT = 0
person.flood_fill_right(x, y, person.resized)
im = person.flood_fill_left(x, y, person.resized)
cv2.imwrite("sidechest.jpeg",im)
sidechest = Measure.PIXEL_COUNT + 4
print(f"Side Chest Pixel count -> {Measure.PIXEL_COUNT + 4}")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Processed image saved successfully in your current working directory.
Cropped image shape - (848, 188, 3)
Resized image shape - (288, 64, 3)
Hip detection rectangle mid-point -> (32, 143)
Hip Pixel count -> 50
Cropped image shape - (848, 188, 3)
Resized image shape - (288, 64, 3)
Side-Chest detection mid-point -> (32, 78)
Side Chest Pixel count -> 53


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
df = pd.read_csv('measure.csv') #file containing pixel count
Gender = 'F'
f=[] # stores total pixel count
for i,j in zip(df['front_hip'],df['side_hip']):
  # calculates total perimeter around the hip
    d = i*2 + j*2 
    f.append(round(d,2)) # rounding off to 2 decimal places

no_of_samples = 6
x = np.array(f) # stores input of pixel count obtained from the image
x = x.reshape(no_of_samples, 1)
y = np.array(df['hip']) # stores the output(actual) measurement
y = y.reshape(no_of_samples, 1)

# Model initialization
regression_model = LinearRegression() #using y=mx+c
# Fit the data(train the model)
regression_model.fit(x, y)
# printing values
print('Slope:' ,regression_model.coef_) #calculating slope
print('Intercept:', regression_model.intercept_)#calculating intercept
# calculating hip size using pixel counts
calculated_hip = fronthip*2 + sidehip*2
# calculating final hip(y) using slope and intercept
final_hip = regression_model.coef_*calculated_hip+regression_model.intercept_
print(f"Final Hip - {final_hip[0]}")
# list containing the calculated pixel counts
calculated = []
for i in x:
  # stores the actual body measurement 
    j = regression_model.coef_*i+regression_model.intercept_
    calculated.append(j)
calculatedf = np.array(calculated)
waist = []
for k in calculatedf:
  if Gender =='F':
  # using waist to hip ratio
    r = k * 0.83
  else:
    r = k * 0.98
  waist.append(r)
#print(np.array(waist))
#print(np.array(df['waist']))
a = np.array(waist) # stores input of pixel count obtained from the image
a = a.reshape(no_of_samples, 1)
b = np.array(df['waist'])#s tores the output(actual) measurement
b = b.reshape(no_of_samples, 1)
if Gender == 'F':
  calculated_waist = final_hip * 0.83 # for female
else:
  calculated_waist = final_hip * 0.98 # for male

# Model initialization
regression_model = LinearRegression()
# Fit the data(train the model)
regression_model.fit(a,b)
# printing values
print('Slope:' ,regression_model.coef_) # calculating slope
print('Intercept:', regression_model.intercept_) # calculating intercept
# calculating final waist(y) using slope and intercept
final_waist = regression_model.coef_*calculated_waist+regression_model.intercept_
print(f"Final waist - {final_waist[0]}")

Slope: [[1.14520674]]
Intercept: [-48.78180704]
Final Hip - [102.38548239]
Slope: [[1.57764593]]
Intercept: [-44.99923778]
Final waist - [89.06903522]


In [15]:
df = pd.read_csv('measure.csv')
f=[]
for i,j in zip(df['front_chest'],df['side_chest']):
  # calculates total perimeter around the chest
    d = i*2 + j*2 
    f.append(round(d,2)) # rounding off to 2 decimal places
no_of_samples = 5
x = np.array(f) # stores input of pixel count obtained from the image
x = x.reshape(no_of_samples, 1)
y = np.array(df['bust']) # stores the output(actual) measurement
y = y.reshape(no_of_samples, 1)

# Model initialization
regression_model = LinearRegression()
# Fit the data(train the model)
regression_model.fit(x, y)
# printing values
print('Slope:' ,regression_model.coef_) # calculating slope
print('Intercept:', regression_model.intercept_) # calculating intercept
# calculating chest size using pixel counts
calculated_chest = frontchest*2 + sidechest*2
# calculating final chest(y) using slope and intercept
final_chest = regression_model.coef_*calculated_chest+regression_model.intercept_
print(final_chest[0])

Slope: [[1.75828358]]
Intercept: [-143.28074627]
[109.91208955]


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
df = pd.read_csv('/content/FinalDatasetMyntra.csv')
df.at[9,'Waist']= '91-95 cm (35.5"-37.5")'
df.head(13)

,Bust,Waist,Hips,US size,UK size,EU size,International
0,"74 cm (29"")","58 cm (23"")","80 cm (31.5"")",0,2,30,XXS
1,"74-77 cm (29""-30"")","58-61 cm (23""-24"")","80-84 cm (31.5""-33"")",0,4,32,XS
2,"78-81 cm (31""-32"")","62-64 cm (24""-25"")","85-89 cm (33""-35"")",2,6,34,XS
3,"82-85 cm (32""-33.5"")","65-68 cm (25""-26.5"")","90-94 cm (35""-37"")",4,8,36,S
4,"86-89 cm (33.5""-35"")","69-72 cm (26.5""-28"")","95-97 cm (37""-38"")",6,10,38,S
5,"90-93 cm (35""-36.5"")","73-77 cm (28""-30"")","98-101 cm (38""-40"")",8,12,40,M
6,"94-97 cm (36.5""-38"")","78-81 cm (30""-32"")","102-104 cm (40""-41"")",10,14,42,M
7,"98-102 cm (38""-40"")","82-85 cm (32""-33.5"")","105-108 cm (41""-42.5"")",12,16,44,L
8,"103-107 cm (40""-42"")","86-90 cm (33.5""-35.5"")","109-112 cm (42.5""-44"")",14,18,46,L
9,"108-113 cm (42""-44.5"")","91-95 cm (35.5""-37.5"")","113-116 cm (44""-45.5"")",16,20,48,XL


In [18]:
bust = float(final_chest[0])
waist = float(final_waist[0])
hips = float(final_hip[0])
sample = {'Bust': bust, 'Waist': waist, 'Hips':hips}

labels = {'XS': 1, 'S': 2, 'M': 3, 'L': 4, 'XL': 5, 'XXL': 6}

for i in range(1, len(df), 2):
    # Special 'XXS' case
    if sample['Bust'] == 74 and sample['Waist'] == 58 and sample['Hips'] == 80:
        print(f"Expected size - XXS")
        break
        
    b_min =  list(map(int, df.loc[i, "Bust"].split()[0].split('-')))[0] 
    b_max =  list(map(int, df.loc[i+1, "Bust"].split()[0].split('-')))[1]
    
    w_min =  list(map(int, df.loc[i, "Waist"].split()[0].split('-')))[0] 
    w_max =  list(map(int, df.loc[i+1, "Waist"].split()[0].split('-')))[1]
    
    h_min =  list(map(int, df.loc[i, "Hips"].split()[0].split('-')))[0]
    h_max =  list(map(int, df.loc[i+1, "Hips"].split()[0].split('-')))[1]
    
    if b_min <= round(sample['Bust'], 0) <= b_max:
        b = df.loc[i, 'International']
    if w_min <= round(sample['Waist'], 0) <= w_max:
        w = df.loc[i, 'International']
    if h_min <= round(sample['Hips'], 0) <= h_max:
        h = df.loc[i, 'International']
    
    if b_min <= sample['Bust'] <= b_max and w_min <= sample['Waist'] <= w_max and h_min <= sample['Hips'] <= h_max:
        print(f"Expected size - {df.loc[i, 'International']}")
        break
else:
  # Average sizecalculation
  avg = round((labels[b] + labels[w] + labels[h])/3,0)
  print(f"Expected size - {list(labels.keys())[list(labels.values()).index(avg)]}")


Expected size - L
